In [1]:
# importing libraries
import os
import pandas as pd
import geopandas as gpd

In [2]:
# setting the working directory - you need to change this to your own directory
#import os
os.chdir('/home/jovyan/work/CUSP_data_dive/Data')
print(os.getcwd())

/home/jovyan/work/CUSP_data_dive/Data


In [3]:
['bge_code', 'bge_type', 'la_name', 'co_name', 'reg_name', 'country', 'total_pop', 'urban_area', 'urban_pct', 'tc_goal', 
'treecanopy', 'tc_gap', 'priority_i', 'inc_rank', 'incnorm', 'inc_dec', 'emp_rank', 'empnorm', 'emp_dec', 'hlth_rank', 'hlthnorm', 
'hlth_dec', 'temp_diff', 'tempnorm', 'NO2_avg', 'PM25_avg', 'apb_index', 'dep_ratio', 'depratnorm', 'dep_perc', 'tes', 'la_tes', 
'pctmineth', 'pct_child', 'pct_senior', 'la_code', 'co_code', 'peat', 'geometry']

['bge_code',
 'bge_type',
 'la_name',
 'co_name',
 'reg_name',
 'country',
 'total_pop',
 'urban_area',
 'urban_pct',
 'tc_goal',
 'treecanopy',
 'tc_gap',
 'priority_i',
 'inc_rank',
 'incnorm',
 'inc_dec',
 'emp_rank',
 'empnorm',
 'emp_dec',
 'hlth_rank',
 'hlthnorm',
 'hlth_dec',
 'temp_diff',
 'tempnorm',
 'NO2_avg',
 'PM25_avg',
 'apb_index',
 'dep_ratio',
 'depratnorm',
 'dep_perc',
 'tes',
 'la_tes',
 'pctmineth',
 'pct_child',
 'pct_senior',
 'la_code',
 'co_code',
 'peat',
 'geometry']

In [4]:
cols= ['bge_code', 'bge_type', 'la_name', 'reg_name', 'hlthnorm', 'treecanopy', 'incnorm', 'empnorm', 'apb_index', 'pct_senior', 'pctmineth', 'geometry']

In [5]:
# read in the shapefile
tree_canopies_gpd = gpd.read_file('england_tes.shp', columns=cols)

print(f"Shapefile data frame is {tree_canopies_gpd.shape[0]:,} x {tree_canopies_gpd.shape[1]}")

Shapefile data frame is 27,979 x 12


In [6]:
# only retain London data
tree_canopies_gpd = tree_canopies_gpd[tree_canopies_gpd['reg_name'] == 'London']
print(tree_canopies_gpd.shape)
print(tree_canopies_gpd.head())

(4986, 12)
   apb_index   bge_code bge_type   empnorm  hlthnorm   incnorm  \
0   0.882636  E01000001     LSOA  0.003112  0.022240  0.000396   
1   0.882636  E01000002     LSOA  0.050427  0.095570  0.089695   
2   0.870083  E01000003     LSOA  0.539933  0.464112  0.436091   
3   0.890412  E01000005     LSOA  0.761287  0.454759  0.816219   
4   0.640963  E01000006     LSOA  0.339323  0.342859  0.572662   

                la_name  pct_senior  pctmineth reg_name  treecanopy  \
0        City of London    0.250169   0.206242   London    0.108141   
1        City of London    0.200290   0.222222   London    0.041181   
2        City of London    0.179234   0.261166   London    0.126761   
3        City of London    0.093236   0.618529   London    0.042198   
4  Barking and Dagenham    0.085452   0.678049   London    0.088654   

                                            geometry  
0  POLYGON ((-10582.361 6713754.192, -10580.94 67...  
1  POLYGON ((-9805.39 6713828.13, -9809.844 67138...  


In [7]:
# drop irrelevant columns
tree_canopies_gpd = tree_canopies_gpd.drop(columns=['bge_type', 'la_name', 'reg_name'])

In [8]:
# generate a pandas data frame by dropping the geometry column
tree_canopies = tree_canopies_gpd.drop(columns='geometry')
print(f"Data frame 1 is {tree_canopies.shape[0]:,} x {tree_canopies.shape[1]}")

Data frame 1 is 4,986 x 8


In [9]:
SMHI = pd.read_csv('MHI_London_22.csv', encoding="utf-8")
print(f"Data frame 2 is {SMHI.shape[0]:,} x {SMHI.shape[1]}")

Data frame 2 is 4,651 x 2


In [10]:
BHRatio = pd.read_csv('Bad_health_ratio.csv', encoding="utf-8")
print(f"Data frame 3 is {BHRatio.shape[0]:,} x {BHRatio.shape[1]}")

Data frame 3 is 4,994 x 2


In [11]:
edu_level4 = pd.read_csv('edu_level4.csv', encoding="utf-8")
print(f"Data frame 4 is {edu_level4.shape[0]:,} x {edu_level4.shape[1]}")

Data frame 4 is 4,994 x 2


Do a data merge to add the additional variables.

In [12]:
# merging BHRatio with tree_canopies data frame using a left join
reg_bh = tree_canopies.merge(BHRatio, left_on='bge_code', right_on='LSOA_code', how='left')
reg_bh = reg_bh.drop(columns=['LSOA_code'])
print(reg_bh.tail())
print(f"New Data frame is {reg_bh.shape[0]:,} x {reg_bh.shape[1]}")

      apb_index   bge_code   empnorm  hlthnorm   incnorm  pct_senior  \
4981   0.759742  E01035718  0.000702  0.001660  0.000625    0.100624   
4982   0.795291  E01035719  0.443228  0.069857  0.484766    0.108405   
4983   0.795634  E01035720  0.443228  0.069857  0.484766    0.096248   
4984   0.768645  E01035721  0.800412  0.480365  0.771722    0.138758   
4985   0.787506  E01035722  0.050412  0.078357  0.207570    0.175421   

      pctmineth  treecanopy   BHRatio  
4981   0.383476    0.364934  0.015570  
4982   0.380503    0.113101  0.035461  
4983   0.349219    0.188865  0.051767  
4984   0.443729    0.104089  0.062366  
4985   0.348236    0.298477  0.041273  
New Data frame is 4,986 x 9


In [13]:
# merging Edu_level4 with reg_bh data frame using a left join
reg_bh_edu = reg_bh.merge(edu_level4, left_on='bge_code', right_on='LSOA_code', how='left')
reg_bh_edu = reg_bh_edu.drop(columns=['LSOA_code'])
print(reg_bh_edu.tail())
print(f"New Data frame is {reg_bh_edu.shape[0]:,} x {reg_bh_edu.shape[1]}")

      apb_index   bge_code   empnorm  hlthnorm   incnorm  pct_senior  \
4981   0.759742  E01035718  0.000702  0.001660  0.000625    0.100624   
4982   0.795291  E01035719  0.443228  0.069857  0.484766    0.108405   
4983   0.795634  E01035720  0.443228  0.069857  0.484766    0.096248   
4984   0.768645  E01035721  0.800412  0.480365  0.771722    0.138758   
4985   0.787506  E01035722  0.050412  0.078357  0.207570    0.175421   

      pctmineth  treecanopy   BHRatio   Edu_L4+  
4981   0.383476    0.364934  0.015570  0.470486  
4982   0.380503    0.113101  0.035461  0.662443  
4983   0.349219    0.188865  0.051767  0.575729  
4984   0.443729    0.104089  0.062366  0.545635  
4985   0.348236    0.298477  0.041273  0.656716  
New Data frame is 4,986 x 10


In [14]:
# merging MHI data with reg_bh_edu data frame using a left join
final_df = reg_bh_edu.merge(SMHI, left_on='bge_code', right_on='lsoa11', how='left')
final_df = final_df.drop(columns=['lsoa11'])
print(final_df.tail())
print(f"New Data frame is {final_df.shape[0]:,} x {final_df.shape[1]}")

      apb_index   bge_code   empnorm  hlthnorm   incnorm  pct_senior  \
4981   0.759742  E01035718  0.000702  0.001660  0.000625    0.100624   
4982   0.795291  E01035719  0.443228  0.069857  0.484766    0.108405   
4983   0.795634  E01035720  0.443228  0.069857  0.484766    0.096248   
4984   0.768645  E01035721  0.800412  0.480365  0.771722    0.138758   
4985   0.787506  E01035722  0.050412  0.078357  0.207570    0.175421   

      pctmineth  treecanopy   BHRatio   Edu_L4+  MHI_2022  
4981   0.383476    0.364934  0.015570  0.470486       NaN  
4982   0.380503    0.113101  0.035461  0.662443       NaN  
4983   0.349219    0.188865  0.051767  0.575729       NaN  
4984   0.443729    0.104089  0.062366  0.545635       NaN  
4985   0.348236    0.298477  0.041273  0.656716       NaN  
New Data frame is 4,986 x 11


In [15]:
# export the final data frame to a csv file
final_df.to_csv('regression_tree_health.csv', index=False)